In [1]:
import pandas as pd
import numpy as np
import os
import re
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

C:\Users\gbao\AppData\Local\Continuum\anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
path = r"C:\Users\gbao\Google 云端硬盘\BUDT758T\Project\Data"
data_dict = pd.read_excel(os.path.join(path,"data_dictionary_2020.xlsx"),index_col=0)
X_train = pd.read_csv(os.path.join(path,"airbnb_train_x.csv"),index_col=0)
y_train = pd.read_csv(os.path.join(path,"airbnb_train_y.csv"),index_col=0)
X_test = pd.read_csv(os.path.join(path,"airbnb_test_x.csv"),index_col=0)
X_all = pd.concat([X_train,X_test],keys=["train","test"])
#whole_data = X_whole.merge(y_train,how="left",left_index=True,right_index=True)
test_index = X_test.index
print(X_train.shape)
print(X_test.shape)
HPI = pd.read_excel(os.path.join(path,"HPI_AT_BDL_county.xlsx"), skiprows=6)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
train_index = X_train.index
val_index = X_val.index

HPI = HPI.groupby(["FIPS code"]).max()

C:\Users\gbao\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2,4,6,7,8,10,11,28,35,43,45,69) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(100000, 69)
(12208, 69)


In [3]:
ohe_features = ["bed_type","cancellation_policy","experiences_offered",
                "host_has_profile_pic","host_identity_verified","host_is_superhost",
               "host_response_time","host_neighbourhood","host_verifications",
               "instant_bookable","is_business_travel_ready","is_location_exact",
               "jurisdiction_names","license","market","neighbourhood",
               "property_type","require_guest_phone_verification","require_guest_profile_picture",
               "requires_license","room_type","smart_location"]

features = ["amenities","bed_type","cancellation_policy",
                "host_has_profile_pic","host_identity_verified","host_is_superhost",
               "host_response_time","host_verifications",
               "instant_bookable","is_business_travel_ready","is_location_exact",
               "property_type","require_guest_phone_verification","require_guest_profile_picture",
               "requires_license","room_type"]



In [4]:
for i in features:
    unique_element = pd.unique(X_train[i])
    print(i, len(unique_element))
    print(unique_element)
    print("-----------------------------------------------------")

amenities 69278
['{TV,"Cable TV",Internet,Wifi,"Air conditioning",Kitchen,Heating,Washer,Dryer,"Smoke detector","Carbon monoxide detector"}'
 '{TV,"Cable TV",Internet,Wifi,"Air conditioning",Kitchen,"Buzzer/wireless intercom",Heating,Washer,Dryer,"Smoke detector","Carbon monoxide detector",Essentials,"Lock on bedroom door","translation missing: en.hosting_amenity_49","translation missing: en.hosting_amenity_50"}'
 '{TV,"Cable TV","Wireless Internet","Air conditioning",Kitchen,Heating,Washer,Dryer,"Smoke detector",Essentials,Shampoo,Hangers,Iron,"Laptop friendly workspace","Self Check-In","Smart lock","Hot water","Bed linens",Microwave,"Coffee maker",Refrigerator,"Dishes and silverware",Oven,Stove,"Long term stays allowed"}'
 ...
 '{Wifi,"Air conditioning",Heating,"Hair dryer","translation missing: en.hosting_amenity_49","translation missing: en.hosting_amenity_50"}'
 '{Internet,"Wireless Internet","Air conditioning",Kitchen,"Free parking on premises","Indoor fireplace",Heating,Washer,D

In [5]:
#data processing function
def data_process(data, features):
    data["amenities"] = data.amenities.str.split(",").apply(len)
    quantile_25 = data.amenities.quantile(0.25)
    quantile_75 = data.amenities.quantile(0.75)
    data["amenities"] = data.amenities.apply(lambda x:"Low" if x<quantile_25 else ("Hihg" if x>quantile_25 else "Middel"))
    data["host_verifications"] = data.host_verifications.astype("str")
    data["host_verifications"] = data.host_verifications.str.split(",")
    data["host_verifications"] = data["host_verifications"].apply(len)
    quantile_25 = data.host_verifications.quantile(0.25)
    quantile_75 = data.host_verifications.quantile(0.75)
    data["host_verifications"] = data.host_verifications.apply(lambda x:"Low" if x<quantile_25 else ("Hihg" if x>quantile_25 else "Middel"))
    
    valid_element = ['Real Bed','Futon','Couch','Pull-out Sofa','Airbed']
    data.bed_type = data.bed_type.apply(lambda x: np.nan if x not in valid_element else x)
    valid_element = ['strict','moderate','flexible','super_strict_30','super_strict_60']
    data.cancellation_policy = data.cancellation_policy.apply(lambda x: np.nan if x not in valid_element else x)
    valid_element = ['t','f']
    data.host_has_profile_pic = data.host_has_profile_pic.apply(lambda x: np.nan if x not in valid_element else x)
    data.host_identity_verified = data.host_identity_verified.apply(lambda x: np.nan if x not in valid_element else x)
    data.host_is_superhost = data.host_is_superhost.apply(lambda x: np.nan if x not in valid_element else x)
    data.instant_bookable = data.instant_bookable.apply(lambda x: np.nan if x not in valid_element else x)
    data.is_business_travel_ready = data.is_business_travel_ready.apply(lambda x: np.nan if x not in valid_element else x)
    data.is_location_exact = data.is_location_exact.apply(lambda x: np.nan if x not in valid_element else x)
    data.require_guest_phone_verification = data.require_guest_phone_verification.apply(lambda x: np.nan if x not in valid_element else x)
    valid_element = ['within a few hours','within an hour','within a day','a few days or more']
    data.host_response_time = data.host_response_time.apply(lambda x: np.nan if x not in valid_element else x)
    
    pattern = "(\d{5})"
    data.zipcode = data.zipcode.astype("str", copy=False)
    data.zipcode = data.zipcode.str.extract(pattern)
    data.zipcode = data.zipcode.astype("str", copy=False)
    data.zipcode = data.zipcode.apply(lambda x: x[:4])
    
    numerical_column = data_dict[data_dict["variable type"] == "numerical"].index.values
    
    pattern = "(\d+\.*\d*)"
    for i in numerical_column:
        temp = data[i].astype("str")
        temp = temp.str.extract(pattern)
        data[i] = temp.astype("float")
        
    temp = features+list(numerical_column)
    data = data[temp]

    #data.dropna(thresh=15, inplace=True)
    for i in features:
        temp = pd.get_dummies(data[i], prefix = i)
        data.drop([i],axis=1,inplace=True)
        data = pd.concat((data, temp), axis=1)
    return(data)


X_all = data_process(X_all, features)
X_train = X_all.loc["train"]
X_train.dropna(thresh=15, inplace=True)
X_test = X_all.loc["test",:]

C:\Users\gbao\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\gbao\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
def xgb_data_process(X_train_processed, y_train):
    temp = X_train_processed.merge(y_train,how='left',left_index=True,right_index=True)
    temp.dropna(subset=["high_booking_rate"],inplace=True)
    columns=y_train.columns
    y_train_xgb = temp.high_booking_rate
    X_train_xgb = temp.drop(columns=columns)

    dtrain = xgb.DMatrix(X_train_xgb, label=y_train_xgb)
    return(dtrain)

dtrain = xgb_data_process(X_train, y_train)
#dval = xgb_data_process(data_process(X_val, features), y_val)
param = {"max_depth": 100, "eta":1, "objective":"binary:logistic"}
num_round = 1000
bst = xgb.train(param, dtrain, num_round)

In [7]:
dtest = xgb.DMatrix(X_test)
ypred = bst.predict(dtest)

In [11]:
result = pd.Series(map(lambda x: 1 if x>0.5 else 0, ypred),name="high_booking_rate")
len(result)
result.to_excel("high_booking_rate.xlsx")                   

In [23]:
total_gain = bst.get_score(importance_type="total_gain")
total_gain 

{'host_response_time_within an hour': 7439.274164421536,
 'host_response_time_within a few hours': 594.6713529635006,
 'host_is_superhost_f': 1346.773816728298,
 'longitude': 5271.751016809148,
 'amenities_Low': 186.12082733760687,
 'minimum_nights': 3783.296822606266,
 'security_deposit': 1939.8586792035198,
 'host_response_rate': 1072.189564702688,
 'extra_people': 1387.4199388365118,
 'availability_60': 4058.4989609682157,
 'availability_30': 3342.4296962534763,
 'latitude': 4698.626058478646,
 'availability_90': 2587.0264855060864,
 'guests_included': 691.7205032947005,
 'accommodates': 876.0636075340449,
 'cleaning_fee': 3897.3320063643023,
 'monthly_price': 486.5872550478127,
 'price': 3732.6737097375844,
 'bedrooms': 544.3362639855127,
 'host_listings_count': 2016.417933865017,
 'maximum_nights': 1686.2988006608284,
 'host_identity_verified_t': 18.366721624828017,
 'availability_365': 3867.9902104154257,
 'amenities_Hihg': 239.94891318170647,
 'host_acceptance_rate': 529.6964767